In [5]:
import numpy as np
import pandas as pd

import os
import random

import warnings

warnings.filterwarnings(action='ignore')

path = '/opt/ml/input/data/'
train = pd.read_csv(path + 'train_data.csv')
test = pd.read_csv(path + 'test_data.csv')

k = 100 # k개까지만 문제 기록을 남기려고함.

In [3]:
# 어짜피 시간순으로 정렬되어 있음. 시간 역순으로 정렬함.
dat = pd.concat([train, test])
dat = dat.loc[::-1]
dat.reset_index(drop= True, inplace= True)
#dat = dat.sort_values(by = ['userID', 'Timestamp'], ascending= False)
dat['tem'] = 1
dat['seq'] = dat.groupby('userID')['tem'].apply(lambda x : x.cumsum())

test_id = test['userID'].unique()
test = dat[(dat['userID'].isin(test_id)) & (dat['seq'] <= k)]
train = dat[~((dat['userID'].isin(test_id)) & (dat['seq'] <= k))]
train['seq'] = train.apply(lambda x : x['seq'] - k  if x['userID'] in test_id else x['seq'], axis = 1)

user_num = dat['userID'].nunique()
valid_data = train[train['seq'] <= k]
train_new = train[train['seq'] > k]

train_datas = train_new[train_new['seq'] <= k*2]
train_new = train_new[train_new['seq'] > k*2]
train_new['userID'] = train_new['userID'] + user_num

for i in range(3, 1860 // k + 2):
    train_data = train_new[train_new['seq'] <= k * i]
    train_new = train_new[train_new['seq'] > k * i]
    train_datas = pd.concat([train_datas, train_data])
    train_new['userID'] = train_new['userID'] + user_num

In [4]:
train_datas['seq'] = (train_datas['seq']-1) % k + 1
valid_data['seq'] = (valid_data['seq']-1) % k + 1
test['seq'] = (test['seq']-1) % k + 1

train_datas.sort_values(['userID', 'seq'], ascending= [True,False], inplace= True)
valid_data.sort_values(['userID', 'seq'], ascending= [True,False], inplace= True)
test.sort_values(['userID', 'seq'], ascending= [True,False], inplace= True)

tem = valid_data.drop_duplicates(['userID'], keep='first')
lst = tem[tem['seq'] <= 14]['userID'].unique()
valid = valid_data[~(valid_data['userID'].isin(lst))]

tem = train_datas.drop_duplicates(['userID'], keep='first')
lst = tem[tem['seq'] <= 14]['userID'].unique()
train = train_datas[~(train_datas['userID'].isin(lst))]

train.drop(['tem'], axis = 1).to_csv(path + 'GKT/train_data.csv', index = False)
valid.drop(['tem'], axis = 1).to_csv(path + 'GKT/valid_data.csv', index = False)
test.drop(['tem'], axis = 1).to_csv(path + 'GKT/test_data.csv', index = False)